<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

大規模なゲノムデータを用いた解析: ゲノムワイド関連解析(GWAS)
---

　今回から、ゲノム情報解析の一例として、大規模なゲノムデータセットを用いた解析を勉強・体験していきます。

大規模ゲノムデータ解析シリーズ: 
- GWAS: Genome wide association study(GWAS)
- Genomic Prediction入門 : Genomic Predictionとは, 機械学習の基本, 統計モデル, 育種への応用...等々

また、今回(GWAS)のテキストのみ**PythonではなくR**を用いて行います。このノートのコードセルではRが動きます。

今回扱うGWASや統計解析に関してはPythonよりRの方がパッケージ等充実しているためです。(次回以降はPythonに戻ります。)

# 今回の勉強内容
　このテキストでは、大規模ゲノムデータ解析シリーズの第1回として、GWASについて勉強します。

　内容は以下の通りです。

1. GWASとは？
  1. 遺伝的変異と形質の関係
  1. GWASの概要
  1. GWASの解析手順
1. 必要となるデータセットの準備
  1. 遺伝子型データと形質データ
1. GWAS入門
  1. 単純な検定でGWASをやってみる
  1. 検定の限界
  1. 代表的な統計モデルの紹介(ろじすてぃっくも)
  1. イネを用いた実例
  1. サンプル数やSNP数の影響
1. GWASが行えるソフトウェアの紹介
1. GxE効果


---

　一度次のコードセルを実行して、実習に必要なライブラリのインストールや、プログラム、サンプルファイルのダウンロードしてください。

In [ ]:
system("mkdir library")
.libPaths("library")
install.packages("rrBLUP")
devtools::install_github("variani/lme4qtl")
tar("library.tar.gz", "library")

In [ ]:
################################
##  以下の実習の前に一度実行してください。##
################################
## Rパッケージのインストール、プログラム・サンプルファイルのダウンロード
system("wget -q -O library.tar.gz https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/library.tar.gz")
system("wget -q -O GWAS_sample_genotype_rice.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GWAS_sample_genotype_rice.csv")
system("wget -q -O GWAS_sample_phenotype_rice.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GWAS_sample_phenotype_rice.csv")
system("wget -q -O GWAS_sample_genotype.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GWAS_sample_genotype.csv")
system("wget -q -O GWAS_sample_phenotype.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GWAS_sample_phenotype.csv")
untar("library.tar.gz")
.libPaths("library")
# 確認
library(rrBLUP)
library(lme4qtl)

# 1. GWASとは？

## 1-1. 遺伝的変異と形質の関係
　GWAS(ゲノムワイド関連解析)とは、表現型の多様性に影響を与える遺伝的変異を統計的に探索する事を指します。

QTL-seqの時に、**遺伝子マッピング（Gene mapping）**手法について少し学びました。

　遺伝子マッピングがどのような手法だったかというと、個体AとBのあいだの形質の違いを作っている遺伝子（原因遺伝子）がゲノム上のどこにあるのかを特定する手法です。どのようにおこなうのか？

1. 個体AとBのゲノム配列を比較すると、個体間の塩基の違い（例えば、SNP; Single nucleotide polymorphism）を発見できます。しかし、個体AとBの間の比較だけでは、どの変異が個体間の形質の違いを作っているのかを特定できません。
2. そこで、個体AとBを交配し、ゲノムをシャッフルした個体集団（分離集団）を作ります。
3. 分離集団の各個体について、形質とゲノム上の各マーカーの遺伝子型も調べ、**形質と遺伝子型に連関（Association）**があるゲノム上の場所を探し、原因遺伝子を含むゲノム領域を特定します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/gene_mapping.png?raw=true" alt="gene_mapping" height="280px" align="middle">

　上述のように、分離集団の全個体の遺伝子型を使って遺伝子マッピングする方法を**QTL解析（QTL analysis）**と言いました。QTLとはQuantitative trait locus(量的形質遺伝子座)の略称です。

　QTL-seqは、QTL解析をより省力的にした方法で、次世代シーケンスデータを用いて、分離集団の一部の個体のシーケンス情報から遺伝子マッピングをおこなう手法でした。

　GWASではこのようなQTLの探索を全ゲノム領域に対して網羅的に統計的手法を用いて行うことを指します。

### (参考) QTL-seqの概要

　QTL-seqでは、分離集団のうち極端な形質値をもつ個体のバルクシーケンスデータを用いて遺伝子マッピングをおこないます。バルクシーケンス中に含まれるアリルの割合（SNP-index）の推移をゲノム全体にわたって調べることで、遺伝子の位置を特定します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_overview.png?raw=true" alt="QTL-seq" height="300px" align="middle">

> <small>Takagi, H., Abe, A., Yoshida, K., Kosugi, S., Natsume, S., Mitsuoka, C., Uemura, A., Utsushi, H., Tamiru, M., Takuno, S., Innan, H., Cano, L., Kamoun, S. & Terauchi, R. (2013). [QTL‐seq: rapid mapping of quantitative trait loci in rice by whole genome resequencing of DNA from two bulked populations](https://onlinelibrary.wiley.com/doi/full/10.1111/tpj.12105). The Plant Journal, 74(1), 174-183.</small>

## 1-2. GWASの概要
　GWASでは、上述した分離集団等を用いて、解析に用いられる遺伝的変異それぞれについて、特定の形質に対する線形回帰モデルを考え、回帰分析を行うことが多いです。例えばあるSNPについて、以下の様なモデルを考えます。

$$
y=S\beta+\epsilon
$$

　ここで、yは解析の対象としたい量的形質、Sはひとつの遺伝的変異です。($\epsilon$は残差。)

※今の段階ではごく単純な回帰モデルを例として挙げています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_basic.png?raw=true" alt="gwas_basic" height="600px" align="middle">

　このようなモデルを各遺伝的変異に対してたて、回帰分析を行うことで、遺伝的変異Sに対する効果量（$β$）の推定値（$\hat{β}$）やその標準誤差（$SE(\hat{β})$）、p値などの要約統計量（summary statistics）が得られます。

　このような分析を全領域の各遺伝的変異に対して行い、形質値と相関の強いゲノム領域を探索していきます。

　例えば、全ゲノム領域の遺伝的変異に対して、上述したモデルを用いてそれぞれ独立に回帰分析を行い、遺伝的変異が形質に対して有意な効果があるのかをp値として求めると以下の様な結果が得られます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/manhattan.png?raw=true" alt="manhattan" height="280px" align="middle">

　上図は、すべての遺伝的変異(x軸)に対して独立に回帰分析を行った結果の−log10(p)値（y軸）が示されており、このような図をマンハッタン・プロット(Manhattan plot)と呼びます。

## 1-3. GWASの解析手順

GWASでは、ゲノムをシャッフルした個体集団（分離集団）であったり、様々な異なった遺伝的背景を持つ集団の「形質値」と「遺伝子型」のデータが必要になります。

GWASの流れ:
1. **個体間の交配や、大規模な系統を集めて、異なる遺伝的背景を持つ集団を作る**
2. 集団を栽培し、目的とする形質値を計測する。
3. ゲノム配列（参照配列）を用意する
4. 集団の各個体のシーケンスデータを得る
5. 参照配列にリードをアライメントする
6. SNPを検出し、各個体のSNPの遺伝子型情報を得る。
7. 2.と6.で得た形質値データ、遺伝子型データを元に、統計解析を行いQTLを探索する。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_process.png?raw=true" alt="GWAS_process" height="800px" align="middle">

SNPの検出(VCFファイルなど)に関してはQTL-seqの回でやったので、今回はデータを得られた後の7.の部分をやっていきます。


# 2. 必要となるデータセットの準備

当然ですが、どれだけ大規模で信頼性の高いデータを準備できるかが、GWASで良い結果を得られるかどうかを決めます。

## 2-1. 遺伝子型データ

　基本的に集団のSNPデータになります。

各SNP毎に、SNPの位置・各個体の遺伝子型の情報が入ったテーブルデータです。

QTL-seqの時にはバルクシーケンスを行いましたが、GWASでは各個体ごとにシーケンスを行いSNPを検出します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_sequence.png?raw=true" alt="GWAS_sequence" height="800px" align="middle">

バルクシーケンスでは、2回(形質値が大きい集団と小さい集団)シーケンスを行えば良かったですが、今回は各個体毎にシーケンスを行う必要があるので、集団サイズが大きいほどコストがかかります。しかし、集団サイズが小さいと十分な検定力を得られないという問題もあります。

QTL-seqの時にリード数が少ないと、正しくSNP等の遺伝的変異の情報が読み切れないという話もありましたが、どれだけ研究費がかけられるのか、データのサイズがどのくらい必要なのか、様々なことを考慮して研究計画をたてる必要があります。

### データフォーマットについて

GWASで扱うSNPの遺伝子型のデータは、文字や数字で表す方法が主流ですが、回帰分析を行いやすい様に数字で表す場合が多いです。

AA/AB/BB ... A系統のホモ接合型、AとB系統のヘテロ接合型、Bのホモ接合型

0/1/2 ... 親系統1のホモ接合型、親系統1と2のヘテロ接合型、親系統2のホモ接合型

などなど。

In [ ]:
genotype = read.csv("GWAS_sample_genotype.csv", row.names=1)
genotype

## 2-2. 形質データ

目的となる形質の各個体毎の計測値データです。

ただ生き物の形質値は、遺伝子の影響だけではなく環境の影響などによってぶれる可能性が考えられます。

そこで、極力そのような影響を省くことを考え、複数年の計測データなど、反復を行い得られたデータの平均値が使われることが多いです。

In [ ]:
phenotype = read.csv("GWAS_sample_phenotype.csv", header = TRUE)
phenotype

### 2-2-1. 量的形質と質的形質

形質には**量的形質**と**質的形質**があります。

量的形質は、葉の長さや籾数など、連続的/量的に変化する形質を持つ、複数の遺伝子の効果の総和によって支配されることが多い形質です。また、環境が形質に影響を与える場合が多いです。

質的形質は、花の色や抵抗性の有無など、１つまたは２つ程度の少数の遺伝子で決定されている形質で、不連続な階級に分類できる形質です。

質的な形質に関連のある遺伝的領域を探したい場合、ロジスティック回帰など形質に応じた回帰モデルを使用する必要があります(後述)。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_phenotype.png?raw=true" alt="GWAS_phenotype" height="200px" align="middle">

(参考)
前回まで扱っていたQTL-seqは量的形質を対象にしています。非常に似た手法として、Mut-map法という手法があり、こちらは質的形質を対象としたQTL mapping手法になります。

# 3. GWAS入門

　それではここからは、GWASの過程やモデルについて実際に試しながらもう少し詳しく勉強していきましょう。


## 3-1. 単純な検定でGWASをやってみる

　まずは皆さんの持っている知識を使ってGWASをやってみましょう。

「生物統計学」の講義で**t検定**という検定を学んだと思います。このt検定を使って、

あるSNPの遺伝子型がA系統型なのかB系統型なのかによって平均値に差があるかどうかを検定します。

(色んな前提を無視しているので厳密には正しいt検定の使い方とは言えないのですが、仮でやってみています。)

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_intro.png?raw=true" alt="GWAS_intro" height="600px" align="middle">

まずはある一つのSNPに関して検定してみましょう。

Rでは`t.test`という関数で簡単にt検定をすることが出来ます。

`marker = 1`の部分を`marker = 25`など、1~1000の数字に変えてどのSNPの遺伝子型が形質値に影響を与えて良そうか探してみましょう。

In [ ]:
marker = 1

# make dataset for visualization
data_for_vis <- data.frame(Grain.number = phenotype$Grain.number,
                                         genotype = as.character(t(genotype[genotype$Marker == marker, 4:dim(genotype_data)[2]])))
head(data_for_vis)

# draw boxplot
library(ggplot2)
g <- ggplot(data_for_vis, aes(x = genotype, y = Grain.number, fill = genotype))
g <- g + geom_boxplot()
g <- g + geom_jitter(width=0.2)
plot(g)

# perform t-test
t_test <- t.test(data_for_vis$Grain.number[data_for_vis$genotype == 0], data_for_vis$Grain.number[data_for_vis$genotype == 2])
print(paste("p値 =", t_test$p.value))

次に、この検定を全てのSNPに対して検定してみます。

In [ ]:
p_values <- c()
for (i in 1:dim(genotype)[1]) {
  Marker1_2 <- phenotype$Grain.number[genotype[genotype$Marker == i, 4:dim(genotype_data)[2]] == 2]
  Marker1_0 <- phenotype$Grain.number[genotype[genotype$Marker == i, 4:dim(genotype_data)[2]] == 0]
  p_values <- c(p_values, t.test(Marker1_0, Marker1_2)$p.value)
}
GWAS_t_test <- data.frame(Marker=1:dim(genotype_data)[1], p_values=p_values)
GWAS_t_test

各SNP毎に計算されたp値をマンハッタンプロットとして表示してみましょう。

In [ ]:
# calculate -log10p
GWAS_t_test$minus_log10p <-  -log10(GWAS_t_test$p_values)
GWAS_t_test$q_values <- p.adjust(GWAS_t_test$p_values, method = "BH")
FDR_threshold <- min(GWAS_t_test[GWAS_t_test$q_values < 0.01, ]$minus_log10p)
GWAS_t_test$Chrom <- genotype$Chrom
GWAS_t_test$Position <- genotype$Position

g <- ggplot(GWAS_t_test, aes(x = Marker, y = minus_log10p, color=Chrom))
g <- g + geom_point()
g <- g + geom_hline(yintercept = FDR_threshold, linetype = "dashed")
plot(g)

マンハッタンプロットを見ると250, 500, 750番目のSNPマーカー付近の遺伝子型が形質値と強く関連していそうです。

例えば250番目のSNPの結果を見てみると

In [ ]:
GWAS_t_test[250, ]

この時、SNPの数だけ検定を繰り返しているので、多重検定の問題があります。

FDR(False discovery rate)やBonferroni補正などを使って多重検定の補正を行う必要があります。

マンハッタンプロットではFDR値を破線で示しており、この破線を超える統計値を示す遺伝的領域が、形質と優位に関係していると考えられることになります。

## 3-2. 単純な検定の限界

このように、単純なt検定でも形質に関係していそうな遺伝領域の候補を検出できそうです。

ただ、上述したモデルやt検定では、様々な問題点があります。
* 複数の遺伝領域が影響している場合、着目したSNP以外の遺伝子型の影響を無視してしまっている
* 集団内に環境の違いや実験区域の違いなどがあった場合その要素を考慮できない
* (家畜等でGWASを行う場合)血縁関係などが考慮できない
* ...etc

そこで、着目したSNPの遺伝子型だけでなく、その他の要素も組み込めるより汎用性の高いモデルが必要になってきます。

また、先ほど紹介した質的形質を扱う場合には、適した回帰モデルを使用する必要があります。

次の章で代表的なモデルを見てみましょう。

## 3-3. 代表的な統計モデルの紹介

GWASを行うための統計モデルには様々なものが存在し、今も開発が続いています。

ここでは代表的なモデルの1つとしてMixed Linear Model(MLM, 混合モデル)を紹介します。

また、対象形質が質的形質だった場合に利用することが多いロジスティック回帰についても簡単に紹介します。

(ここでは簡単な紹介だけになるので、興味がある人は勉強してみてください。)

### 3-3-1. 混合モデル

GWASでよく使用されるモデルの1つに混合モデルがあります。特に以下の様なモデルが良く使用されます。

$$
\boldsymbol{y}=W\boldsymbol{v}+X\boldsymbol{\beta}+Z\boldsymbol{u}+\boldsymbol{\epsilon}
$$

$W\boldsymbol{v}$は水準や何らかの処理の効果を固定効果として表しており、$X\boldsymbol{\beta}$は着目したSNPの効果を固定効果として表しています。$Z\boldsymbol{u}$は全ゲノム領域のポリジーン効果を変量効果として表したものになります。

つまり、個体毎の環境の違いや、他のゲノム領域の影響を考慮しつつ、着目したSNPに効果($\hat{\beta}$)があるのかを確認することが出来るモデルになっています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_MLM.png?raw=true" alt="GWAS_MLM" height="450px">

この様なモデルを元に、GWASを行うソフトやプログラムがたくさん開発されています。

(参考) 
* Yu et al., (2006) https://www.nature.com/articles/ng1702
* Zhang et al., (2010) https://www.nature.com/articles/ng.546

#### 固定効果と変量効果

※ 読み飛ばしてOKです。混合モデルの簡単な説明をしています。

混合モデルは固定効果と変量効果を扱うことが出来る柔軟なモデルです。

* 固定効果

固定効果は水準間の比較や効果の検証を目的とする要因を固定効果として扱います。

肥料100g vs 肥料200gの効果の違いや, 投薬の有り無しによる効果の違いなどです。

今回だと、あるSNPがA系統なのかB系統なのかという違いによって生じる効果を固定効果としています。

* 変量効果

変量効果は、効果にばらつきがある、分布を持つ要因を変量効果として扱います。ランダム効果等と言われたりもします。

例えば栽培した圃場の違いの効果をモデルに組み込む場合、圃場の形質への影響はある程度ばらつくことが予想されます。

(同じ圃場Xでも、端っこの方とか真ん中あたりとかでちょっと違いますよね)。

このような場合、圃場の違いは変量効果として扱います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_random.png?raw=true" alt="GWAS_random" height="400px">

* 混合モデル

この様な固定効果と変量効果、両者を含めたモデルを混合モデルといいます。

モデルの各効果の推定をするには、最良線形不偏推定値(Best Linear Unbiased Estimates, BLUE)や最良線形不偏予測値(Best Linear Unbiased Predictor, BLUP)という概念、分散成分の推定に一般的に使用される最尤推定法(Maximum Likelihood, ML)や制限付き最尤法(Restricted Maximum Likelihood, REML)と呼ばれる方法の知識が必要になりますが、今回は統計学の講義では無いので割愛します。

興味がある方は参考文献の佐々木先生の本が詳しいので見てみてください。

今回は上述した手法を実装したプログラムやソフトウェアがあるので、それらを利用して解析していきます。

(参考文献)
* 久保拓弥 (2012) データ解析のための統計モデリング入門 一般化線形モデル・階層ベイズモデル・MCMC (確率と情報の科学) 岩波書店
* 佐々木義之（2007）変量効果の推定とBLUP 法．京都大学学術出版会

#### 混合モデルによるGWASの例

Rのパッケージ`rrBLUP`や`GWASpoly`はこの混合モデルを実装したプログラムになります。

(参考)Endelman group (https://potatobreeding.cals.wisc.edu/software/)

今回は`rrBLUP`を使ってGWASを行ってみましょう。

In [ ]:
library(rrBLUP)
X <- genotype[, 4:dim(genotype)[2]]
M <- X-1
geno <- data.frame(marker=genotype$Marker, chrom=genotype$Chrom, pos=genotype$Position, M, check.names=FALSE)
pheno <- phenotype[ ,c(1,2)]
GWAS_rrBLUP <- GWAS(pheno, geno, plot=FALSE)

In [ ]:
# calculate FDR threshold
q_values <- p.adjust(10 ** (-GWAS_rrBLUP$Grain.number), method = "BH")
FDR_threshold <- min(GWAS_rrBLUP$Grain.number[q_values < 0.01])

# draw manhattan plot
g <- ggplot(GWAS_rrBLUP, aes(x = marker, y = Grain.number, color=chrom))
g <- g + geom_point()
g <- g + geom_hline(yintercept = FDR_threshold, linetype = "dashed")
plot(g)

#### 連鎖について

GWASを行うことで候補となる遺伝領域のピークが検出されたと思います。

大抵、ある１つのSNPなどの遺伝的変異だけが検出されるのではなく、周囲のSNPも高くなっていることが分かります。

これは**連鎖**によって、原因遺伝子の近傍領域も一緒に遺伝されることが多いため生じています。

### 3-4-3. ロジスティック回帰モデル

対象とする形質が質的な形質の場合、通常の回帰分析では扱うことが難しくなります。

特に「白い花か赤い花か」や「抵抗性か感受性か」の様な二値形質
では，線形回帰モデルの代わりにロジスティック回帰モデルが用いられます。

　**ロジスティック回帰（Logisitic regression）**は、線形回帰に少し手を加えて、**あるクラスに分類される確率を求める手法**です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logistic_regression.png?raw=true" alt="logistic_regression" height="400px">

　ロジスティック回帰は次の式で表されるモデルです（詳しくは「ロジスティック回帰の式を詳しく　」を参照）。

$$ z = \beta x + \epsilon $$
$$ p = \frac{1}{1+e^{-z}} $$

- 係数$\beta$をSNPの効果、説明変数$x$をSNPの遺伝子型として表しています。$\epsilon$（epsilon）は誤差（切片）です。
- $p$はあるクラスに属している確率を表しています。$e$はネイピア数（=2.7182...）です。  

　このとき、$0 ≦ p ≦ 1$なので、$-\infty < z < \infty$となり、先ほどまで扱っていた線形モデルの様に扱える様になります。

　この様にロジスティック回帰モデルをたてることで、係数$\beta$や誤差$\epsilon$が求まり、GWASが行えるようになるわけです。

#### ロジスティック回帰の式を詳しく...
<small>*※ 読み飛ばしてOKです。ロジスティック回帰の式の背景を数式を使って説明しています。*</small>

　ロジスティック回帰の式を理解するためには、**オッズ(odds)**から始める必要があります。

　ある事象Aが起こる確率を$p$としたとき、Aが起こらない確率は$1-p$となります。このとき、Aの起こる確率とAが起こらない確率の比は、$$\frac{p}{1-p}$$となります。これがオッズと呼ばれるもので、Aが起こる「見込み」を表しています。

確率である$p$の値は0から1の範囲しかとれませんが、オッズに変換することで$0から\infty$までの範囲に、さらに対数をとることで$-\infty$から$\infty$まで値を取ることが出来ます。

先程確率は0から1までしかとれないので回帰式が作れないという話をしましたが、対数オッズを対象にすると線形回帰で扱えそうな値になりますね。

さて、ここで最終的に知りたいことは、**データから確率$p$を推測する式**です。

$$
p = (SNPの情報などの式)
$$

　オッズの対数（**対数オッズ**）をとり、関数とみなすと、$$f(p) = \log{\frac{p}{1-p}}$$となります。これは、**ロジット関数（logit）**として知られる関数です。

　ロジット関数を逆関数にすると、$f(p)$から$p$を求める関数が得られます。
\begin{align*}
f(p) &= \log{\frac{p}{1-p}} \\
e^{f(p)} &= \frac{p}{1-p} \\
e^{f(p)} - e^{f(p)} p &= p \\
p &= {\frac{e^{f(p)}}{e^{f(p)}+1}} \\ &= \frac{1}{1+e^{-f(p)}}
\end{align*}

　ここで、$f(p) = z$ とすると、ロジスティック回帰のふたつ目の式が得られます。これは、**ロジスティック関数（logistic）**として知られる関数です。$$ p = \frac{1}{1+e^{-z}} $$この関数は、入力値$z$からAが起こる確率$p$を求める関数とみなせます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/logistic_curve.png?raw=true" alt="logistic_curve" height="200px">

　ロジスティック回帰は、線形回帰の式をロジスティック関数の入力$z$につなげたものです。

これは、「データx(今回だとSNPの情報)」を「Aがおこる確率$p$」に変換する式です。

$$f(p) =  z = \beta x + \epsilon $$

$$ p = \frac{1}{1+e^{-(\beta x + \epsilon)}} $$

このあたりに関しても、簡単な説明になっているので、興味のある方は各自調べてみてください。

(参考文献)

* 久保拓弥 (2012) データ解析のための統計モデリング入門 一般化線形モデル・階層ベイズモデル・MCMC (確率と情報の科学) 岩波書店

#### ロジスティック回帰による検定の例

白い花になる個体と、赤い花をつける個体から成る集団がいたとして、この質的形質である花の色と関連している遺伝領域をGWASで探してみます。

`glm`関数等でロジスティック回帰は実装出来ますが、`lme4qtl`というパッケージでは変量効果や固定効果も踏まえたロジスティック回帰にが出来るので、今回はこちらを使用してみます。

$$ z = W\boldsymbol{v}+X\boldsymbol{\beta}+Z\boldsymbol{u}+\boldsymbol{\epsilon} $$
$$ p = \frac{1}{1+e^{-z}} $$

(計算時間がかかるのでいくつかのSNPの結果を見てみます)

In [ ]:
marker = 1

# convert white/red label to 0/1
pheno <- phenotype[ ,c(1,3)]
pheno["Flower.color"] <- lapply(pheno["Flower.color"], gsub, pattern="White", replacement = 0)
pheno["Flower.color"] <- lapply(pheno["Flower.color"], gsub, pattern="Red", replacement = 1)
pheno$Flower.color <- as.integer(pheno$Flower.color)

# read genotype_data & generate variance-covariance matrix
M <- t(geno[, -c(1:3)])
K <- A.mat(M,shrink=FALSE)

# statistical test by logistic regression
SNP_geno <- t(geno[marker, -c(1:3)])
colnames(SNP_geno) <- "SNP"
SNP_geno <- cbind(Line = rownames(SNP_geno), SNP_geno)
tmp_pheno <- merge(pheno, SNP_geno, by="Line")
trait <- "Flower.color"
model1_formula <- as.formula(paste0(trait, " ~ (1|Line)"))
model1 <- relmatGlmer(model1_formula, tmp_pheno, relmat = list(Line = K), family = binomial)
model2_formula <- as.formula(paste0(trait, " ~ SNP + (1|Line)"))
model2 <- relmatGlmer(model2_formula, tmp_pheno, relmat = list(Line = K), family = binomial)
print(model2)
# maximum likelihood test (model1 vs model2)
print(paste("p値 =", anova(model1, model2)[2, 8]))

今回のサンプルデータは780番目のSNPが花の色に強い影響を与えるようなデータになっていました。

In [ ]:
# 780番目のマーカーの遺伝子型毎のクロス集計
pheno$marker780 <- t(geno[780, -c(1:3)]) + 1
summary_table <- data.frame(table(pheno$Flower.color, pheno$marker780))
colnames(summary_table) <- c("Flower.color", "Marker780", "Freq")
summary_table

## 3-4. イネを用いた実例

これまではサンプルデータで手法等を確認していました。

このサンプルデータは籾数に対しては綺麗に250, 500, 750番目のSNPが、花の色には780番目のSNPが検出される様に意図的に作ったデータでした。

ここからは実際のイネの交配集団のデータを用いて、どのような解析結果になるのか確認してみましょう。

### 3-4-1. データの読み込み

今回使うデータは、イネの"ひとめぼれ"という系統と"URASAN1"という系統をかけあわせ、遺伝子型がホモ接合型に固定されるようにF2集団から自家交配を数世代繰り返したRIL(recombinant inbred lines)と呼ばれる集団です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GWAS_RIL.png?raw=true" alt="GWAS_RIL" height="200px">

RIL集団は親系統の遺伝子型がランダムに混ざってホモ接合型で固定されているため、ヘテロ接合の影響(雑種強勢など)を考慮する必要が無く、GWASなどを用いたQTLの探索に非常に強力な材料です。

この集団の遺伝子型情報と表現型情報を読み込みましょう。

まずは遺伝子型情報です。今回の場合は、0 = ひとめぼれ型、2 = URASAN1型、1 = ヘテロを表しています。

In [ ]:
# Genotype情報
rice_genotype <- read.csv("GWAS_sample_genotype_rice.csv")
rice_genotype

次に、表現型値の情報です。

今回は籾数と葉の幅をデータとして取得します。

In [ ]:
rice_phenotype <- read.csv("GWAS_sample_phenotype_rice.csv")
rice_phenotype

### 3-4-2. 混合モデルによるGWAS

先ほどやった様に`rrBLUP`を使って混合モデルによるGWASを行ってみましょう。

`rrBLUP`は複数の形質も同時に解析してくれます。

In [ ]:
library(rrBLUP)
X <- rice_genotype[, 4:dim(rice_genotype)[2]]
M <- X-1
geno <- data.frame(marker=rice_genotype$Marker, chrom=rice_genotype$Chrom, pos=rice_genotype$Position, M, check.names=FALSE)
GWAS_rice <- GWAS(rice_phenotype, geno, plot=FALSE)

### 3-4-3. マンハッタンプロットの描写

籾数に関して

In [ ]:
# calculate FDR threshold for Grain number
q_values_GN <- p.adjust(10 ** (-GWAS_rice$Grain.number), method = "BH")
FDR_threshold_GN <- min(GWAS_rice$Grain.number[q_values_GN < 0.01])
GWAS_rice[order(GWAS_rice$Grain.number, decreasing = T), ][1:5, c(1,2,3,5)]

# draw manhattan plot for Grain number
g <- ggplot(GWAS_rice, aes(x = marker, y = Grain.number, color=chrom))
g <- g + geom_point()
g <- g + geom_hline(yintercept = FDR_threshold_GN, linetype = "dashed")
g <- g + ggtitle("Grain number")
plot(g)

葉の幅に関して

In [ ]:
# calculate FDR threshold for Leaf width
q_values_LW <- p.adjust(10 ** (-GWAS_rice$Leaf.width), method = "BH")
FDR_threshold_LW <- min(GWAS_rice$Leaf.width[q_values_LW < 0.01])
GWAS_rice[order(GWAS_rice$Leaf.width, decreasing = T), ][1:5, c(1,2,3,4)]

# draw manhattan plot for Leaf width
g <- ggplot(GWAS_rice, aes(x = marker, y = Leaf.width, color=chrom))
g <- g + geom_point()
g <- g + geom_hline(yintercept = FDR_threshold_LW, linetype = "dashed")
g <- g + ggtitle("Leaf width")
plot(g)

## 3-5. サンプル数やSNP数の影響

次に、先ほどのイネの交配集団をベースに、集団サイズが小さくなったり、検出されたSNP数が少なくなった場合に、結果がどのように変化するのか確認してみましょう。

In [ ]:
sample_size <- 50 # 10~141
SNP_size <- 25781 # 10~25781	

# extract samples & SNPs
sample_index <- sort(sample(1:dim(rice_phenotype)[1], sample_size, replace=FALSE))
SNP_index <- sort(sample(1:dim(rice_genotype)[1], SNP_size, replace=FALSE))
tmp_genotype <- rice_genotype[SNP_index, ]
tmp_phenotype <- rice_phenotype[sample_index, c(1,3)]

# GWAS
X <- tmp_genotype[, 4:dim(tmp_genotype)[2]]
M <- X-1
geno <- data.frame(marker=tmp_genotype$Marker, chrom=tmp_genotype$Chrom, pos=tmp_genotype$Position, M, check.names=FALSE)
GWAS_rice <- GWAS(tmp_phenotype, geno, plot=FALSE)

# calculate FDR threshold for Grain number
q_values_GN <- p.adjust(10 ** (-GWAS_rice$Grain.number), method = "BH")
FDR_threshold_GN <- min(GWAS_rice$Grain.number[q_values_GN < 0.01])
GWAS_rice[order(GWAS_rice$Grain.number, decreasing = T), ][1:5, ]

# draw manhattan plot for Grain number
g <- ggplot(GWAS_rice, aes(x = marker, y = Grain.number, color=chrom))
g <- g + geom_point()
g <- g + geom_hline(yintercept = FDR_threshold_GN, linetype = "dashed")
g <- g + ggtitle("Grain number")
plot(g)

このように、集団サイズが十分でないと検出力が下がったり、シーケンスが十分に出来なくて検出されたSNPの数が少ないと、候補となる領域が広くなってしまいます。

その結果、GWAS自体は行うことが出来ますが、意味のある結果を検出することは難しくなります。

# 4. GWASが行えるソフトウェアの紹介

GWASが行えるソフトウェアやパッケージ、ライブラリ等は無数にあります。

それぞれ異なったモデルや計算手法、考慮できる要素等があり、目的に応じて最適なものを使用する必要があります。

当然考慮したい特殊な処理区等があれば自作のプログラムで解析するのも選択肢の一つです。

ソフトウェア
* TASSEL ... https://www.maizegenetics.net/tassel
* PLINK ... https://www.cog-genomics.org/plink

Rパッケージ
* rrBLUP ... https://potatobreeding.cals.wisc.edu/software/
* GWASpoly ... https://github.com/jendelman/GWASpoly
* r/qtl ... https://rqtl.org/

Pythonライブラリ
* Hail ... https://hail.is/

等々、探せばもっとたくさんあります。

# まとめ

　今回、GWASの概要と解析の流れをみてきました。

GWASは関心のある形質にどのような遺伝子が影響を与えているのか調べるために、非常によく用いられています。

皆さんの中でも、ゲノム解析を主な研究手法としなくても、今後原因遺伝子の探索のきっかけとしてGWASを行う方はたくさんいると思います。


また、ポピュラーな手法なので、GWASを行うための解析プログラムやパッケージが大量に存在しています。

もちろん適切な手法やモデルをたててGWASを行うことは大切ですが、

この様な大規模なデータを用いた解析の場合は、データのサイズと質が何よりも大事です。

望ましい結果が得られない時に、手法を色々変えて無理やり検出させるような人がいますが、

「どれだけのサイズのデータがあれば自分の見たいものが検出できるのか」という点をまず第１に考える必要があるでしょう。

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>